EE 502 P: Analytical Methods
# 5. Transforms
## Prof. Eric Klavins, Electrical and Computer Engineering
### 25 September, 2019
Copyright &copy; 2019, University of Washington

https://www.mathpages.com/home/kmath640/kmath640.htm

Depending on how much of elementary linear algebra and differential equations we take for granted, the proof of the Cayley-Hamilton theorem can be fairly immediate. For an arbitrary square matrix M with characteristic equation f(z) = 0, the general solution of the linear system  is given by x(t) = eMtx(0). We also know that each component of x(t) individually satisfies the differential operator form of the characteristic equation, so we have f(d/dt)x(t) = 0. Inserting the expression for x(t), we get f(M)eMtx(0) = 0, and since this must be true for any initial conditions x(0), it follows that f(M) = 0, which was to be proven.